In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%matplotlib inline
import  matplotlib.pyplot as plt
import joblib
from sklearn.metrics import mean_absolute_percentage_error
import random
import torch
import os

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(seed=521)

In [ ]:
train= pd.read_excel('./dataset/train.xlsx')
test= pd.read_excel('./dataset/test.xlsx')
Y_columns =['TNeff']
X_columns = [col for col in train.columns if col not in ['time','TNeff']]


x_train=train[X_columns]
x_test=test[X_columns]
y_train=train[Y_columns]
y_test=test[Y_columns]

In [ ]:
# data = pd.concat([train, test], axis=0)
# x=data.iloc[:,1:-1]
# y=data.iloc[:,-1]
# from sklearn.preprocessing import MinMaxScaler
#
# scaler1=MinMaxScaler(feature_range=(0.000000001,1))
# scaler2=MinMaxScaler(feature_range=(0.000000001,1))
# scaler3=MinMaxScaler(feature_range=(0.000000001,1))
# scaler4=MinMaxScaler(feature_range=(0.000000001,1))
#
#
#
# x_train=scaler1.fit_transform(train[X_columns])
# y_train=scaler2.fit_transform(train[Y_columns])
# x_test=scaler3.fit_transform(test[X_columns])
# y_test=scaler4.fit_transform(test[Y_columns])

In [ ]:
# data = pd.concat([train, test], axis=0)
#
# roll_cols = ["Fin","Tin","CODin","TNin","NH3in","DOain","NH4a","NO3a","MLSSa","DOaeff","NH4m","NO3m"]
# for i in [j for j in range(1, 6)] + [12, 24, 48, 60]:
#     # data[[ii+f'_delta_{i}' for ii in roll_cols]] = data[roll_cols].diff(periods=i)
#     data[[ii+f'_mean_{i}' for ii in roll_cols]] = data[roll_cols].rolling(i, min_periods=1).mean()
#
# pd.set_option("display.max_columns", 100)
# save_rate = 0.5
# data = data[data.describe().T.sort_values(by="std", ascending=False, inplace=False).iloc[: int(len(data.columns)*save_rate)].index.tolist()]
#
# train = data.iloc[:train.shape[0]].reset_index(drop=True)
# test = data.iloc[train.shape[0]:].reset_index(drop=True)

In [ ]:
import xgboost as xgb
model =xgb.XGBRegressor(n_estimators=200,max_depth=10) # 逻辑回归

lossmin=1
for i in range(100):
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    loss=mean_absolute_percentage_error(y_test, y_pred)
    print(f'*****************epoch{i} MAPE {loss}')

    if loss<1 and loss<lossmin:
        lossmin=loss
        joblib.dump(model, './mpkl/m8_xgboost_.pkl')

model= joblib.load('./mpkl/m8_xgboost_.pkl')
y_pred = model.predict(x_test)
loss=mean_absolute_percentage_error(y_test, y_pred)
print(f'***************************************************')
print('MAPE:', loss)
y_train_pred=model.predict(x_train)

In [ ]:
# y_train=scaler2.inverse_transform(y_train)
# y_train_pred=scaler2.inverse_transform(y_train_pred)
#
#
# y_test=scaler4.inverse_transform(y_test)
# y_pred=scaler4.inverse_transform(y_pred)

In [ ]:
import seaborn as sns
sns.set(context='paper', style='ticks')

fig,axes=plt.subplots(2,1,figsize=(20,10))
ax1=axes[0]
ax2=axes[1]
fig.subplots_adjust(left=0.1, right=0.8, bottom=0.1, top=0.9, wspace=0.2, hspace=0.3)


# x=train_index[::32]
# y1=y_train['AQ'][::32]
# y2=y_train_pred['AQ'][::32]
lns1=ax1.plot(y_train[:],label='y_train',marker='v',markevery=1, color='lightcoral')
lns2=ax1.plot(y_train_pred[:],label='y_train_pred',marker='D',markevery=1, color='dodgerblue')
ax1.tick_params(labelsize=12)
# ax1.set_xlim(0,14208,2000)
# ax1.set_ylim(0,35000, 5000)
# ax1.set_xlabel('Timeindex(8h)',size=20)
# ax1.set_ylabel('AQ(m³/h)',size=20)
ax1.set_title('m1_ridge',size=20,fontweight='bold',loc='left')
lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc=1,fontsize=12,ncol=2,frameon=False)



lns1=ax2.plot(y_test[:],label='y_test',marker='v',markevery=1, color='lightcoral')
lns2=ax2.plot(y_pred[:],label='y_test_pred',marker='D',markevery=1, color='dodgerblue')
ax2.tick_params(labelsize=12)
# ax1.set_xlim(0,14208,2000)
# ax1.set_ylim(0,35000, 5000)
# ax1.set_xlabel('Timeindex(8h)',size=20)
# ax1.set_ylabel('AQ(m³/h)',size=20)
ax2.set_title('m1_ridge',size=20,fontweight='bold',loc='left')
lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax2.legend(lns, labs, loc=1,fontsize=12,ncol=2,frameon=False)




In [ ]:
frame=pd.DataFrame()
frame['ytrain']=pd.DataFrame(y_train_pred)
frame['ytest']=pd.DataFrame(y_pred)
frame.to_excel('./modelresult/m08_xgboost.xlsx')